In this notebook, we'll look at solving inverse problems for PDEs.
This is a well-studied subject but a problem that often goes ignored is what kind of observational data go into it.
If the observational data are spatially dense then they can be interpolated to the finite element mesh, and you're free to pretend as if the measurements are a nice continuous field.
We'll consider the particular problem of using measurements of the solution $u$ of the Poisson problem

$$-\nabla\cdot k\nabla u = q$$

to estimate the right-hand side $q$.
This can be formulated as the problem of finding a critical point of the functional

$$L =
\underbrace{\frac{1}{2}\int_\Omega\left(\frac{u - u^o}{\sigma}\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx}_{\text{regularization}} +
\underbrace{\int_\Omega\left(k\nabla u\cdot\nabla\lambda - q\lambda\right)dx}_{\text{physics constraint}}$$

where $u^o$ are the observational data, and we've introduced a Lagrange multiplier $\lambda$.
This formulation is really nice because the model-data misfit term is an integral that we can easily express in UFL.

But the observational data might be sparse compared to the resolution of the finite element grid, in which case interpolating to a finite element basis might be completely inappropriate.
In that case the model-data misfit has to be written as a finite sum of evaluations at the measurement points $\{x_n\}$:

$$E = \sum_n\frac{|u(x_n) - u^o(x_n)|^2}{2\sigma(x_n)^2}.$$

This might be more correct, but it's much more difficult to express easily in UFL.

### Setting up the problem

To define the inverse problem, we'll need to:

1. Create the right-hand side $q$
2. Solve the Poisson equation to get the true value of the field $u$
3. Define a point cloud $\{x_k\}$ for where the measurements occur
4. Synthesize some observations $u^o_k = u(x_k) + \sigma\zeta_k$ where $\sigma$ is the standard deviation of the synthetic measurement error and each $\zeta_k$ is standard normal

In [ ]:
import firedrake
import firedrake_adjoint
mesh = firedrake.UnitSquareMesh(32, 32)
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

The right-hand side $q$ will be a random trigonometric series.

In [ ]:
from firedrake import Constant, cos, sin
import numpy as np
from numpy import pi as π
from numpy import random

seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q_true = firedrake.Function(Q)
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])
        
        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z
        
        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)
        
        q_true += mode

Ooh pretty

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(q_true, axes=axes, shading='gouraud')
fig.colorbar(colors);

Now let's generate the true solution of the PDE.

In [ ]:
from firedrake import inner, grad, dx
u = firedrake.Function(V)
J = (0.5 * inner(grad(u), grad(u)) - q_true * u) * dx
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = firedrake.derivative(J, u)
firedrake.solve(F == 0, u, bc)
u_true = u.copy(deepcopy=True)

Sort of pretty I guess I dunno

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u, axes=axes, cmap='twilight', shading='gouraud')
fig.colorbar(colors);

Now let's make the observational data.

In [ ]:
num_points = 25
δs = np.linspace(-0.5, 2, num_points + 1)
X, Y = np.meshgrid(δs, δs)
xs = np.vstack((X.flatten(), Y.flatten())).T

θ = π / 12
R = np.array([
    [np.cos(θ), -np.sin(θ)],
    [np.sin(θ), np.cos(θ)]
])

xs = np.array([
    x for x in (xs - np.array([0.5, 0.5])) @ R
    if (0 <= x[0] <= 1) and (0 <= x[1] <= 1)
])

We'll assume the measurements have a signal-to-noise ratio of 20; you can tweak this.

In [ ]:
U = u_true.dat.data_ro[:]
u_range = U.max() - U.min()
signal_to_noise = 20
σ = firedrake.Constant(u_range / signal_to_noise)
ζ = generator.standard_normal(len(xs))
u_obs = np.array(u_true.at(xs)) + float(σ) * ζ

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
axes = fig.add_subplot(projection='3d')
firedrake.trisurf(u, axes=axes, alpha=0.25, cmap='twilight')
axes.scatter(xs[:, 0], xs[:, 1], u_obs, color='black');

Now we'll create a point cloud object and a function space $Z$ on this point cloud.

In [ ]:
point_cloud = firedrake.VertexOnlyMesh(mesh, xs)
Z = firedrake.FunctionSpace(point_cloud, 'DG', 0)
u_o = firedrake.Function(Z)
u_o.dat.data[:] = u_obs

### Calculating derivatives

Now let's see what pyadjoint can and can't do.

In [ ]:
u = firedrake.Function(V)
q = firedrake.Function(Q)
J = (0.5 * inner(grad(u), grad(u)) - q * u) * dx
F = firedrake.derivative(J, u)
firedrake.solve(F == 0, u, bc)

In [ ]:
Πu = firedrake.interpolate(u, Z)
E = 0.5 * ((u_o - Πu) / σ)**2 * dx

In [ ]:
α = firedrake.Constant(0.5)
R = 0.5 * α**2 * inner(grad(q), grad(q)) * dx

Because `E` and `R` are defined over different domains -- `E` on the point cloud and `R` on the entire mesh -- we can't do `assemble(E + R)`.
And yet this seems to work; ours is not to question why.

In [ ]:
J = firedrake.assemble(E) + firedrake.assemble(R)

In [ ]:
q̂ = firedrake_adjoint.Control(q)
Ĵ = firedrake_adjoint.ReducedFunctional(J, q̂)

### Minimizing the objective

We get a NotImplementedError if we try this with Newton-CG.
For now this uses BFGS, we should just make it talk to ROL.

In [ ]:
q_min = firedrake_adjoint.minimize(Ĵ, method='BFGS')

Possibly a little over-regularized but it looks ok if you take your glasses off.

In [ ]:
fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True)
for ax in axes:
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)

kw = {'vmin': -5, 'vmax': +5, 'shading': 'gouraud'}
axes[0].set_title('Estimated')
firedrake.tripcolor(q_min, axes=axes[0], **kw)
axes[1].set_title('True')
firedrake.tripcolor(q_true, axes=axes[1], **kw);